# Project Code and Data Walkthrough

### Duncan Park and Kyle Parran

#### This notebook intends to give users a brief tour of the data pulled, processed, and analyzed in our project.



#




## Import Statements

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# local modules
import sys
sys.path.append('../src/')
from calc_format_futures_data import *
from pull_futures_data import *

import warnings
warnings.filterwarnings("ignore")

Loading library list...
Done


## Loading Data from WRDS

#### First, for each commodity you try to find info on all the futures contracts for that respective commodity

In [2]:
info_df = fetch_wrds_contract_info(2036, 'paper')
info_df

,futcode,contrcode,contrname,contrdate,startdate,lasttrddate
0,46.0,2036.0,ORANGE JUICE (FCOJ-A),0303,2001-10-08,2003-03-11
1,189.0,2036.0,ORANGE JUICE (FCOJ-A),0300,1998-09-08,2000-03-13
2,190.0,2036.0,ORANGE JUICE (FCOJ-A),0704,2002-12-03,2004-07-12
3,391.0,2036.0,ORANGE JUICE (FCOJ-A),0986,1985-04-01,1986-09-17
4,912.0,2036.0,ORANGE JUICE (FCOJ-A),0188,1986-08-01,1988-01-18
...,...,...,...,...,...,...
201,318247.0,2036.0,ORANGE JUICE (FCOJ-A),0978,1977-10-03,1978-09-18
202,318248.0,2036.0,ORANGE JUICE (FCOJ-A),1174,1973-12-03,1974-11-15
203,318249.0,2036.0,ORANGE JUICE (FCOJ-A),1175,1975-01-20,1975-11-14
204,318250.0,2036.0,ORANGE JUICE (FCOJ-A),1176,1975-12-01,1976-11-16


#### Then, for each commodity, we can query all the time series data for all of these relevant futures.

In [3]:
futcodes = tuple(info_df["futcode"].unique())
futcodes_contrdates = info_df.set_index("futcode")["contrdate"].to_dict()

In [4]:
futcodes_contrdates

{46.0: '0303',
 189.0: '0300',
 190.0: '0704',
 391.0: '0986',
 912.0: '0188',
 1453.0: '0382',
 1927.0: '0301',
 2629.0: '0189',
 3039.0: '0102',
 3705.0: '0505',
 5272.0: '0792',
 5676.0: '0508',
 5890.0: '0980',
 8169.0: '1101',
 8809.0: '1105',
 8816.0: '1199',
 8920.0: '0182',
 9186.0: '0999',
 9893.0: '0187',
 11306.0: '1185',
 11333.0: '0394',
 13260.0: '0101',
 16213.0: '0994',
 17358.0: '0984',
 17468.0: '0589',
 18000.0: '0195',
 18122.0: '0993',
 19091.0: '0380',
 19652.0: '0983',
 22147.0: '0503',
 26208.0: '0906',
 26217.0: '0988',
 28059.0: '1103',
 29326.0: '0388',
 29421.0: '0106',
 29559.0: '0705',
 30857.0: '0989',
 34316.0: '1100',
 34348.0: '0584',
 34837.0: '0192',
 35537.0: '0100',
 36371.0: '0900',
 37099.0: '0979',
 38438.0: '0995',
 40694.0: '0389',
 41620.0: '0308',
 42274.0: '0702',
 43334.0: '0981',
 45718.0: '1184',
 52491.0: '0791',
 52630.0: '0901',
 54269.0: '0987',
 58985.0: '0193',
 61315.0: '0586',
 61689.0: '0304',
 61853.0: '1179',
 63091.0: '0593',

In [5]:
data_contracts = fetch_wrds_fut_contract(futcodes_contrdates, 'paper')
data_contracts

,futcode,date_,settlement,contrdate
0,46.0,2001-10-08,93.250000,0303
1,46.0,2001-10-09,93.250000,0303
2,46.0,2001-10-10,93.549988,0303
3,46.0,2001-10-11,93.799988,0303
4,46.0,2001-10-12,98.799988,0303
...,...,...,...,...
72477,318251.0,1977-11-10,124.500000,1177
72478,318251.0,1977-11-11,125.500000,1177
72479,318251.0,1977-11-14,129.850000,1177
72480,318251.0,1977-11-15,131.100000,1177


In [6]:
data_contracts = data_contracts.sort_values(["futcode", "date_"])

# Group by futcode and the month-year of date_, then take the last row in each group
monthly_df = data_contracts.groupby(["futcode", data_contracts["date_"].dt.to_period("M")]).tail(1)

In [7]:
monthly_df

,futcode,date_,settlement,contrdate
17,46.0,2001-10-31,99.500000,0303
36,46.0,2001-11-30,103.299990,0303
54,46.0,2001-12-28,98.049988,0303
75,46.0,2002-01-31,92.899994,0303
94,46.0,2002-02-28,91.000000,0303
...,...,...,...,...
72405,318251.0,1977-07-29,111.950000,1177
72428,318251.0,1977-08-31,109.100000,1177
72449,318251.0,1977-09-30,127.900000,1177
72470,318251.0,1977-10-31,125.750000,1177


In [15]:
res = extract_prompt_and_12m_settlements(monthly_df)
res

,prompt_settlement,settlement_12m
date_,,
1973-08-01,NaN,NaN
1973-09-01,NaN,NaN
1973-10-01,NaN,NaN
1973-11-01,NaN,NaN
1973-12-01,NaN,NaN
...,...,...
2008-07-01,NaN,NaN
2008-08-01,108.549990,NaN
2008-09-01,NaN,NaN


In [ ]:
monthly_df = data_contracts.groupby("month").apply(extract_prompt_and_12m_settlements, include_groups=False).reset_index()


In [ ]:
stats = compute_futures_stats(monthly_df)